In [1]:
import pandas as pd
import numpy as np


In [2]:
# file paths and names
path_input = '/Users/srilakshmi/Desktop/Thesis/Data/District-level/Totals/'
name_input_file = 'cropland-total.csv'

path_output = '/Users/srilakshmi/Desktop/Thesis/Treatment-Control-Assignment/Perc-Flooded-NRT/Cropland/'
name_output_file = 'Perc_Cropland_Flooded.csv'


In [3]:
# clean total cropland data

total = pd.read_csv(path_input+name_input_file, header=0)

colnames = []
for col in total.columns:
    colnames.append(col)

total = total.dropna(subset=[colnames[1]])
total = total[[colnames[1], colnames[5], colnames[6]]]
total = total.rename(columns={colnames[1]: "Region",
                              colnames[5]: "Flooded",
                              colnames[6]: "Total"})

total['Perc_Flooded'] = list(total['Flooded']/total['Total'])

total = total.fillna(0)

total = total.replace([np.inf, -np.inf], np.nan)

total = total.drop(columns=['Total', 'Flooded'])
total = total.sort_values(by=['Perc_Flooded'])

total

# save file to disk

total.to_csv(path_output+name_output_file, index = False, header=True)


In [5]:
total.head(100)

,Region,Perc_Flooded
215,Kadjebi,0.000000
207,Bia West,0.000000
208,Jaman South,0.000000
209,Abura / Asebu / Kwamankese,0.000000
214,Cape Coast Metro,0.000000
...,...,...
85,Atiwa,0.000373
92,Keta Municipal,0.000395
86,Tarkwa Nsuaem,0.000406
116,Birim North,0.000417
